In [1]:
import mne
import numpy as np
import pandas as pd
import os

# Parameters
n_samples_per_miniepoch = 60
n_samples_ini = 20
n_miniepochs_per_trial = 60
n_negative_miniepochs = 4  # How far back to go
bids_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids'
subjects = [f"S{i:02}" for i in range(1, 21)]
specific_condition = '0067_Left' 

for subject in subjects:
    print(f"\n=== Processing {subject} ===")
    fn_movie = f'sub-{subject}_Dynamic_preprocessed-epo.fif'
    movie_path = os.path.join(bids_dir, 'derivatives/preprocessed', fn_movie)

    if not os.path.exists(movie_path):
        print(f"File not found: {movie_path}")
        continue

    epochs_movie = mne.read_epochs(movie_path)
    epochs_movie = epochs_movie[epochs_movie.metadata['condition'] != 'catch']

    # Index alignment
    onset_first_miniepoch = np.where(epochs_movie.times == 0)[0][0] + n_samples_ini
    miniepochs = []

    # Metadata
    condition = epochs_movie.metadata['condition']
    direction = [c.split('_')[1] for c in condition]
    right_idx = [d == 'Right' for d in direction]
    left_idx = [d == 'Left' for d in direction]
    degree = [int(c.split('_')[0]) for c in condition]
    run = epochs_movie.metadata['run_nr']

    # Negative timepoints
    for i in range(n_negative_miniepochs):
        onset_end = onset_first_miniepoch - i * n_samples_per_miniepoch
        onset_start = onset_end - n_samples_per_miniepoch
        me = mne.EpochsArray(
            epochs_movie._data[:, :, onset_start:onset_end],
            info=epochs_movie.info
        )
        df = pd.DataFrame()
        df['condition'] = condition
        df['direction'] = direction
        df['degrees'] = degree
        df['timepoints'] = -(i + 1)
        df['trialnum'] = np.arange(len(condition))
        df['run'] = run
        me.metadata = df
        miniepochs.append(me)

    # Positive timepoints
    for i in range(n_miniepochs_per_trial):
        onset = onset_first_miniepoch + i * n_samples_per_miniepoch
        me = mne.EpochsArray(
            epochs_movie._data[:, :, onset:onset + n_samples_per_miniepoch],
            info=epochs_movie.info
        )
        deg = np.zeros(len(degree))
        deg[right_idx] = np.array(degree)[right_idx] + 3 * i
        deg[left_idx] = np.array(degree)[left_idx] - 3 * i
        deg[deg > 360] -= 360
        deg[deg < 0] += 360

        df = pd.DataFrame()
        df['condition'] = condition
        df['direction'] = direction
        df['degrees'] = deg
        df['timepoints'] = i
        df['trialnum'] = np.arange(len(condition))
        df['run'] = run
        me.metadata = df
        miniepochs.append(me)

    # Concatenate all
    mini_epochs = mne.concatenate_epochs(miniepochs)
    metadata_df = mini_epochs.metadata

    # Sort metadata and print subset
    sorted_metadata = metadata_df.sort_values(by=['condition', 'timepoints'])
    filtered_metadata = sorted_metadata[sorted_metadata['timepoints'].isin([-2])]
    print("Filtered metadata at timepoint -2:")
    print(filtered_metadata)

    if specific_condition in sorted_metadata['condition'].values:
        condition_data = sorted_metadata[sorted_metadata['condition'] == specific_condition]
        degrees = np.array(condition_data['degrees'])
        print(f"Degrees for condition {specific_condition}: {np.unique(degrees)}")
    else:
        print(f"Condition {specific_condition} not found in {subject}")

    # Save
    out_path = os.path.join(bids_dir, 'derivatives/preprocessed', f'sub-{subject}_miniEpochsBaselineIncluded_preprocessed-epo.fif')
    mini_epochs.save(out_path, overwrite=True)



=== Processing S01 ===
Reading /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Dynamic_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Reading /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S01_Dynamic_preprocessed-epo-1.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
504 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 6 columns
Not setting metadata
448 matching eve